In [6]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# AON TEAM: CBU Coding Challenge - Features Engineering Pipeline
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# ==============================================================================
# 0. ПОДГОТОВКА СРЕДЫ
# ==============================================================================
import pandas as pd
import numpy as np
import os
import warnings
from google.colab import drive
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
print("✅ Среда подготовлена.")


✅ Среда подготовлена.


In [7]:
# ==============================================================================
# 1. ЗАГРУЗКА ДАННЫХ
# ==============================================================================
drive.mount('/content/drive', force_remount=True)
DATA_PATH = '/content/drive/MyDrive/DATA for CBU'
INPUT_FILE = os.path.join(DATA_PATH, 'alldata.csv') # Берем идеально чистый файл из EDA

try:
    df = pd.read_csv(INPUT_FILE)
    print(f"✅ Очищенные данные '{os.path.basename(INPUT_FILE)}' успешно загружены. Форма: {df.shape}")
except FileNotFoundError:
    print(f"❌ ФАЙЛ НЕ НАЙДЕН по пути: {INPUT_FILE}")
    df = pd.DataFrame() # Создаем пустой DataFrame, чтобы код не упал

Mounted at /content/drive
✅ Очищенные данные 'alldata.csv' успешно загружены. Форма: (89999, 38)


In [9]:
# ==============================================================================
# 2. ПАЙПЛАЙН ГЕНЕРАЦИИ "ЗОЛОТЫХ" ПРИЗНАКОВ
# ==============================================================================

def generate_golden_features(df):
    """
    Принимает очищенный DataFrame и обогащает его "золотыми" признаками,
    основанными на бизнес-логике и взаимодействиях.
    """
    if df.empty:
        print("⚠️ DataFrame пуст, генерация признаков пропущена.")
        return df

    print("\n--- 🚀 Начало генерации 'золотых' признаков ---")

    df_eng = df.copy()
    epsilon = 1e-6

    # --- (1) Продвинутые Финансовые Взаимодействия (Income vs Lifestyle) ---
    print("  - Генерация: Продвинутые финансовые коэффициенты...")
    df_eng['income_to_loan_ratio'] = df_eng['annual_income'] / (df_eng['loan_amount'] + epsilon)
    df_eng['debt_to_credit_limit_ratio'] = df_eng['total_debt_amount'] / (df_eng['total_credit_limit'] + epsilon)
    df_eng['payment_to_debt_ratio'] = df_eng['monthly_payment'] / (df_eng['total_debt_amount'] + epsilon)
    df_eng['income_utilization_for_debt'] = df_eng['annual_debt_payment'] / (df_eng['annual_income'] + epsilon)

    # --- (2) Поведенческие и Демографические Прокси-Признаки ---
    print("  - Генерация: Поведенческие и демографические прокси...")
    df_eng['age_to_credit_history_ratio'] = df_eng['age'] / (df_eng['oldest_credit_line_age'] + epsilon)
    df_eng['debt_per_dependent'] = df_eng['total_debt_amount'] / (df_eng.get('num_dependents', 0) + 1)

    # --- (3) Кластеризация для создания сегментов клиентов (Unsupervised Learning) ---
    print("  - Генерация: Сегментация клиентов (KMeans)...")
    cluster_features = ['age', 'annual_income', 'credit_score', 'debt_to_income_ratio', 'total_credit_limit', 'loan_amount']
    cluster_features = [f for f in cluster_features if f in df_eng.columns]

    df_cluster_data = df_eng[cluster_features].fillna(df_eng[cluster_features].median())

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_cluster_data)

    kmeans = KMeans(n_clusters=6, random_state=42, n_init='auto')

    # Сначала создаем pd.Series, а потом меняем тип. Это более надежный способ.
    cluster_labels = kmeans.fit_predict(scaled_data)
    df_eng['customer_segment'] = pd.Series(cluster_labels, index=df_eng.index).astype('category')

    # --- (4) Взаимодействия Топ-Признаков из EDA ---
    print("  - Генерация: Взаимодействия топ-признаков...")
    if 'credit_score' in df_eng.columns and 'debt_to_income_ratio' in df_eng.columns:
        df_eng['cr_score_x_dti_ratio'] = df_eng['credit_score'] * df_eng['debt_to_income_ratio']
    if 'age' in df_eng.columns and 'annual_income' in df_eng.columns:
        df_eng['age_x_income'] = df_eng['age'] * df_eng['annual_income']
    if 'interest_rate' in df_eng.columns and 'loan_term' in df_eng.columns:
        df_eng['interest_x_term'] = df_eng['interest_rate'] * df_eng['loan_term']

    df_eng.replace([np.inf, -np.inf], np.nan, inplace=True)

    print("\n✅ Генерация 'золотых' признаков завершена!")
    return df_eng
# Запускаем пайплайн генерации
df_golden = generate_golden_features(df)


--- 🚀 Начало генерации 'золотых' признаков ---
  - Генерация: Продвинутые финансовые коэффициенты...
  - Генерация: Поведенческие и демографические прокси...
  - Генерация: Сегментация клиентов (KMeans)...
  - Генерация: Взаимодействия топ-признаков...

✅ Генерация 'золотых' признаков завершена!


In [12]:
# ==============================================================================
# 3. ФИНАЛИЗАЦИЯ И СОХРАНЕНИЕ
# ==============================================================================
if not df_golden.empty:
    print("\n--- 🚀 Финализация и сохранение обогащенного датасета ---")

    new_features_count = len(df_golden.columns) - len(df.columns)

    print(f"Исходное количество столбцов: {len(df.columns)}")
    print(f"Добавлено новых признаков: {new_features_count}")
    print(f"Итоговое количество столбцов: {len(df_golden.columns)}")

    new_columns = [col for col in df_golden.columns if col not in df.columns]
    print("\nСгенерированные признаки:")
    print(new_columns)

    # Сохраняем итоговый датасет для следующего этапа - моделирования
    OUTPUT_FILE = os.path.join(DATA_PATH, 'alldata_engineered.csv')
    df_golden.to_csv(OUTPUT_FILE, index=False)

    print(f"\n✅ Обогащенный датасет сохранен по пути: '{OUTPUT_FILE}'")
    print("Этот файл готов для загрузки в ваш пайплайн OOF-ансамбля.")


--- 🚀 Финализация и сохранение обогащенного датасета ---
Исходное количество столбцов: 38
Добавлено новых признаков: 10
Итоговое количество столбцов: 48

Сгенерированные признаки:
['income_to_loan_ratio', 'debt_to_credit_limit_ratio', 'payment_to_debt_ratio', 'income_utilization_for_debt', 'age_to_credit_history_ratio', 'debt_per_dependent', 'customer_segment', 'cr_score_x_dti_ratio', 'age_x_income', 'interest_x_term']

✅ Обогащенный датасет сохранен по пути: '/content/drive/MyDrive/DATA for CBU/alldata_engineered.csv'
Этот файл готов для загрузки в ваш пайплайн OOF-ансамбля.
